1. Loading data for reviews and metadata

In [1]:
import torch

torch.cuda.empty_cache()

In [2]:
torch.cuda.mem_get_info()[0] / torch.cuda.mem_get_info()[1]

0.07941210074733189

In [3]:
from datasets import load_dataset

# Load the reviews dataset for Gift Cards
reviews = load_dataset(
    "McAuley-Lab/Amazon-Reviews-2023",
    name="raw_review_Video_Games",
    split="full",
    trust_remote_code=True
)

In [4]:
# Load the metadata dataset for Gift Cards
metadata = load_dataset(
    "McAuley-Lab/Amazon-Reviews-2023",
    name="raw_meta_Video_Games",
    split="full",
    trust_remote_code=True
)

In [5]:
reviews_df = reviews.to_pandas()
metadata_df = metadata.to_pandas()

In [6]:
reviews_df = reviews_df[['text', 'parent_asin']]
metadata_df = metadata_df[['title', 'parent_asin']]

2. Processing data(use parent_asin to link reviews and metadata)

In [7]:
import pandas as pd

In [8]:
processed_dataset = pd.merge(reviews_df, metadata_df, on='parent_asin', how='inner')

In [9]:
processed_dataset = processed_dataset[['title', 'text']]

3. Testing for data format

4. Import language model

In [10]:
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer

# Define arguments for loading the model
class args:
    model_name_or_path = "EleutherAI/gpt-neo-125m"
    cache_dir = "./cache/"
    model_revision = "main"
    use_fast_tokenizer = True

# Load model configuration
config = AutoConfig.from_pretrained(
    args.model_name_or_path,
    cache_dir=args.cache_dir,
    revision=args.model_revision,
    use_auth_token=None,  # No authentication token required for this public model
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    args.model_name_or_path,
    cache_dir=args.cache_dir,
    use_fast=args.use_fast_tokenizer,
    revision=args.model_revision,
    use_auth_token=None,
)

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    args.model_name_or_path,
    from_tf=bool(".ckpt" in args.model_name_or_path),  # Handle TensorFlow checkpoints if applicable
    config=config,
    cache_dir=args.cache_dir,
    revision=args.model_revision,
    use_auth_token=None,
).to("cuda")

# Print model summary
print(f"Model loaded: {args.model_name_or_path}")
print(f"Model parameters: {model.num_parameters()}")


Model loaded: EleutherAI/gpt-neo-125m
Model parameters: 125198592


5. Prepare and check final dataset

In [11]:
# Assign padding token if not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [12]:
from datasets import Dataset

def get_data(df) -> dict:
    # transform dataframe into dataset
    data_dict = Dataset.from_pandas(
        df
    ).train_test_split(
        0.1
    ).map(
        format_item
    ).remove_columns(
        ["title", "text"]
    )
    
    return data_dict

# format question-answer pair into content to be fed in to model
def format_item(item) -> dict:
    product = item["title"].strip()
    review = item["text"].strip()
    return { "content": f"Product: {product}\nReview: {review}" }

data_dict = get_data(processed_dataset)

Map:   0%|          | 0/4162153 [00:00<?, ? examples/s]

Map:   0%|          | 0/462462 [00:00<?, ? examples/s]

6. Train model

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

# Customize training arguments
training_arguments = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="adamw_torch",
    save_steps=1000,
    logging_steps=100,
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    max_steps=50000,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    gradient_checkpointing=True,
)

# Initialize trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=data_dict['train'],
    eval_dataset=data_dict['test'],
    dataset_text_field="content",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_arguments,
)

# Start training
trainer.train()


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/scratch/bchk/aguha/venv/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOK

Map:   0%|          | 0/4162153 [00:00<?, ? examples/s]

Map:   0%|          | 0/462462 [00:00<?, ? examples/s]

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
100,10.260500
200,9.749400
300,9.340100
400,9.390300
500,9.199100
600,9.286600
700,8.994000
800,9.011200
900,8.995400
1000,9.047100


In [ ]:
example_inputs = tokenizer([f"Product: Final Fantasy VII: Remake - PlayStation 4\nReview:"], return_tensors="pt").to(model.device)
example_outputs = model.generate(
    **example_inputs, 
    max_length=150,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
    use_cache=True,
    pad_token_id=tokenizer.eos_token_id)
output_text = tokenizer.decode(example_outputs[0])
print(output_text)